In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

import sqlite3
import re

# Парсинг сайта и настройка базы данных

**План:**
- скачаем тексты песен с помощью парсера
     - *выберем сайт, посмотрим его структуру*
     - *скачаем страницу*
     - *найдем ссылки и скачаем тексты*
     - *сохраним в переменные*
- сохраним данные в БД



## 1. Парсинг

### Первый исполнитель

In [3]:
url = 'https://textypesen.com/krovostok/' 

songs_list = requests.get(url)
#songs_list.status_code # проверка исправности

#songs_list.content
songs_soup = BeautifulSoup(songs_list.content, 'html.parser') # переведем результат в более читаемый вид

songs_list = []

for line in songs_soup.find_all('a'):  # вытащим из html адреса с текстами и добавим их в список
    if str(line).find('krovostok') != -1:
        song_url = 'https://textypesen.com' + line['href']
        songs_list.append(song_url)

In [4]:
songs_list

['https://textypesen.com/krovostok/baklany/',
 'https://textypesen.com/krovostok/belyj-jaguar/',
 'https://textypesen.com/krovostok/besporjadki/',
 'https://textypesen.com/krovostok/biografija/',
 'https://textypesen.com/krovostok/britni/',
 'https://textypesen.com/krovostok/byt-plokhim/',
 'https://textypesen.com/krovostok/vesna/',
 'https://textypesen.com/krovostok/gantelja/',
 'https://textypesen.com/krovostok/genetika/',
 'https://textypesen.com/krovostok/gidropon-dunul/',
 'https://textypesen.com/krovostok/glaza/',
 'https://textypesen.com/krovostok/derevnja/',
 'https://textypesen.com/krovostok/dumaj-pozitivno/',
 'https://textypesen.com/krovostok/dush/',
 'https://textypesen.com/krovostok/dushno/',
 'https://textypesen.com/krovostok/zhest/',
 'https://textypesen.com/krovostok/zjoma/',
 'https://textypesen.com/krovostok/zimnjaja/',
 'https://textypesen.com/krovostok/intro/',
 'https://textypesen.com/krovostok/kisa/',
 'https://textypesen.com/krovostok/kolkhozniki/',
 'https://tex

In [5]:
# функция очистки текста от лишних символов
def replace_all(text, to_replace):
    for symbol in to_replace:
        text = text.replace(symbol, ' ')
    return text

In [6]:
parsed_songs = {}

for song in songs_list:
    song_url = requests.get(song)
    song_soup = BeautifulSoup(song_url.content, 'html.parser')
    song_text = str(song_soup.find_all('p')[1]) # находим нужную часть с текстом
    song_text = replace_all(song_text, ['<br/>', '</br>', '<br>', '<p class="font-size-20">', '</p>', '\xa0', '\n' ])
    song_text = song_text.rstrip().lstrip() # убираем лишние пробелы в начале и в конце
    parsed_songs[song] = song_text
    time.sleep(1) # создает интервал между действиями, чтобы нас не забанили

In [96]:
parsed_songs.keys()

dict_keys(['https://textypesen.com/krovostok/baklany/', 'https://textypesen.com/krovostok/belyj-jaguar/', 'https://textypesen.com/krovostok/besporjadki/', 'https://textypesen.com/krovostok/biografija/', 'https://textypesen.com/krovostok/britni/', 'https://textypesen.com/krovostok/byt-plokhim/', 'https://textypesen.com/krovostok/vesna/', 'https://textypesen.com/krovostok/gantelja/', 'https://textypesen.com/krovostok/genetika/', 'https://textypesen.com/krovostok/gidropon-dunul/', 'https://textypesen.com/krovostok/glaza/', 'https://textypesen.com/krovostok/derevnja/', 'https://textypesen.com/krovostok/dumaj-pozitivno/', 'https://textypesen.com/krovostok/dush/', 'https://textypesen.com/krovostok/dushno/', 'https://textypesen.com/krovostok/zhest/', 'https://textypesen.com/krovostok/zjoma/', 'https://textypesen.com/krovostok/zimnjaja/', 'https://textypesen.com/krovostok/intro/', 'https://textypesen.com/krovostok/kisa/', 'https://textypesen.com/krovostok/kolkhozniki/', 'https://textypesen.com

### Второй исполнитель

In [8]:
url = 'https://txtsong.ru/xaski/'
songs_list = requests.get(url)
songs_soup = BeautifulSoup(songs_list.content, 'html.parser')

songs_list = []

for line in songs_soup.find_all('a'):  # вытащим из html адреса с текстами и добавим их в список
    if str(line).find('xaski/') != -1:
        song_url = line['href']
        songs_list.append(song_url)
        
songs_list = list(set(songs_list)) # удалим дубли

parsed_songs_2 = {}

for song in songs_list:
    song_url = requests.get(song)
    song_soup = BeautifulSoup(song_url.content, 'html.parser')
    song_text = str(song_soup.find_all('p')[4]) # находим нужную часть с текстом
    song_text = replace_all(song_text, ['<p>', '<br/>', '</br>', '<br>', '</p>', '\n', '<!-- Yandex.RTB R-A-1318229-28 --> <div id="yandex_rtb_R-A-1318229-28"></div> <script>window.yaContextCb.push(()=>{   Ya.Context.AdvManager.render({     renderTo: \'yandex_rtb_R-A-1318229-28\',     blockId: \'R-A-1318229-28\'   }) })</script>' ])    
    song_text = song_text.rstrip().lstrip() # убираем лишние пробелы в начале и в конце
    parsed_songs_2[song] = song_text
    time.sleep(1) # создает интервал между действиями, чтобы нас не забанили

In [98]:
parsed_songs_2.keys()

dict_keys(['https://txtsong.ru/xaski/haski-pesnya-dlya-k/', 'https://txtsong.ru/xaski/haski-npc/', 'https://txtsong.ru/xaski/xaski-lyucifer/', 'https://txtsong.ru/xaski/xaski-marmelad/', 'https://txtsong.ru/xaski/xaski-white-punk-pauche-moloko/', 'https://txtsong.ru/xaski/xaski-maslo-chernogo-tmina-ubej-menya/', 'https://txtsong.ru/xaski/xaski-aferist/', 'https://txtsong.ru/xaski/ozyora-xaski-luna/', 'https://txtsong.ru/xaski/rich-xaski-son-esera/', 'https://txtsong.ru/xaski/xaski-vladyka-slov/', 'https://txtsong.ru/xaski/xaski-beng-beng/', 'https://txtsong.ru/xaski/xaski-piroman-17/', 'https://txtsong.ru/xaski/zavet-xaski-igly-i-ogni/', 'https://txtsong.ru/xaski/xaski-feat-maslo-chyornogo-tmina-tartarary/', 'https://txtsong.ru/xaski/sovet-98-haski-marmelad-xxii/', 'https://txtsong.ru/xaski/xaski-chernym-cherno/', 'https://txtsong.ru/xaski/xaski-rich-tarakanij-bog/', 'https://txtsong.ru/xaski/xaski-krot/', 'https://txtsong.ru/xaski/xaski-panelka/', 'https://txtsong.ru/xaski/xaski-iuda/

## 2. Работа с БД

In [10]:
con = sqlite3.connect('songs.db') # создаем локальную базу данных

In [11]:
cur = con.cursor()
cur.execute('CREATE TABLE song_texts(author, text, url, date)') # создаем таблицу
res = cur.execute('SELECT name FROM sqlite_master')
res.fetchone() # проерим создалась ли таблица

for k, v in parsed_songs.items():   # перебираем ключи и значения и записываем их в таблицу

    sql_insert = """
                    INSERT INTO song_texts (author, text, url, date)
                    VALUES ('{}', '{}', '{}', '{}')
                    """.format('Krovostok', v, k, datetime.today())

    res = cur.execute(sql_insert)
    con.commit()
    
res = cur.execute('SELECT * FROM song_texts')
result = res.fetchall()

In [12]:
for k, v in parsed_songs_2.items():   # перебираем ключи и значения и записываем их в таблицу

    sql_insert = """
                    INSERT INTO song_texts (author, text, url, date)
                    VALUES ('{}', '{}', '{}', '{}')
                    """.format('Haski', v, k, datetime.today())

    res = cur.execute(sql_insert)
    con.commit()

res = cur.execute('SELECT * FROM song_texts')
result = res.fetchall()

In [13]:
result[-1]

('Haski',
 'Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино, в котором  Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино, в котором  Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино, в котором  Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино     Я вернулся в твою голову как в старые ботинки (husky)  Суперзвезды умирают понарошку  Выхожу в окно будто на красную дорожку  Окруженный пиздюками как какой-нибудь кюре  Я купаюсь в лести, как в картофельном пюре  В одиночном кинозале через дырки в черепной  Я наблюдаю ебанутый новый день очередной     В котором  Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино, в котором  Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино, в котором  Я смотрю кино про себя, в котором  Я смотрю кино про себя, в котором  Я смотрю кино, в котором  Я смотрю 

План:

- подготовим текст
- скачаем готовую модель
- подготовим окружение для работы
- дообучим модель

In [34]:
to_replace = ['!', '?', '.', ',', ':', '-', '  ', '[', ']', '))', '"', '(', ')', '— ', '  ', '\u2005', 'х5', 'х2', ' «', '» ']

In [102]:
# убираем имя исполнителя
text_songs = [i[1] for i in result]
text_prep_all = []

In [103]:
for i in text_songs:
    text_prep = re.findall('[А-Я][^А-Я]*', i)
    print(text_prep)
    print(len(text_prep))
    
    if len(text_prep) < 5:
        pass
    else:
        text_prep = [a.lower() for a in text_prep]
        text_prep = [replace_all(a, to_replace) for a in text_prep]
        text_prep = [a.rstrip().rstrip() for a in text_prep]
        text_prep = [a.lstrip() for a in text_prep]
        print(text_prep)
    
        clean_text = []
        for idx, val in enumerate(text_prep):
            if text_prep[idx] in [' ', '']:
                pass
            else:
                clean_text.append(val)

        text_prep = []
        for val in clean_text:
            if len(val) <= 10:
                pass
            else:
                text_prep.append(val)

        text_prep_unique = list(set(text_prep))
        text_prep_str = '\n'.join(text_prep_unique)
        print('text_prep_str:', len(text_prep_str))

        if len(text_prep_str) < 1:
            pass
        else:
            text_prep_all.append(text_prep_str)
            print('text_prep_all:', len(text_prep_str))

        #if len(text_prep_str) > 0:
        #    text_prep_all.append(text_prep_str)
   
        

   
    
    
    
    
   # text_prep_all.append(text_prep_str)

['Бакланы жрут коробками план, ву танг клан, хлам  ', 'Нy его в спам, прошлогодний снег заебал он всех,  ', 'Но речь не о том, дорасскажу потом  ', 'Стою за гаражами в аляске, прижали к стене,  ', 'Но греет на кармане полоска острой стали  ', 'Короче приближаться стали ко мне  ', 'Ну я, слышь ты,  ', 'Розовый бройлер на анаболиках  ', 'Такие как мы наркоманы бля и алкоголики  ', 'Дают оборотку таким как ты ', 'Хуём тебе по подбородку красавица  ', 'Скалится мудачок  ', 'Бросается типа убрать пытается,  ', 'Но удар убойный  ', 'Как бешек бокс в два косых забитый  ', 'И переносица в мозг переносится,  ', 'Хрипит он и чавкает  ', 'Не нюхнуть колумбийский кокс  ', 'Бодяженный сахарной пудрой  ', 'И ещё заебатая приблуда  ', 'Второму который поменьше  ', 'Исправлен прикус  ', 'Избавлен малыш от золотых фикс  ', 'Покушает мягкого муда  ', 'И вот по глине козлик катается  ', 'Тот кто среди них был за ', 'Тайсона  ', 'Как пиздатый микс, свелась история  ', 'Стало попросторнее  ', 'Кость в кост

In [100]:
text_prep_all[8:12]

['и я их ебурю ебурю ебурб ебурю ебурю ебурю подручными средствами и в снег и в бурю\nизбавил всех от этой злоебучей женщины плохая наследственность в этом не ее вины\nфаршмачок мамулю я порционно слил в унитаз я сделал это не для себя я сделал это для вас\nменя опередила ее дочь тобишь моя мать но я с рождения знал что она та еще блядь\nмногим не понять что я делаю на этой планете многим не понять а мне похуй вы мне поверьте\nмногие не знают вообще что они сами тут делают а я знаю точно и в этом смысле я их всех делаю\nпотом пыталась меня внучка своего захуярить что бы я ее не захуярил когда смогу захуярить\nбогу теперь один я совсем остался отучился работаю в генетике разобрался\nи что ее надо валить при первом же случае вот такой у нас род весь злоебучий\nее мать моя бабуленька была тоже тем еще дьяволом захуярила моего батю за то что он назвал ее дьяволом\nя знаю точно точнее мне кажется некуда надо ебурить насмерть каждого недруга\nя шинканул ее на мелкий фаршмачок\nмне мама с дет

In [104]:
len(text_prep_all)

109

In [60]:
text_prep_unique

['я смотрю кино в котором',
 'суперзвезды умирают понарошку',
 'маленький и теплый будто кала коробок',
 'помни нас дохуя что червей в бомже',
 'окруженный пиздюками как какой нибудь кюре',
 'слезы на брекетах прыщавого подростка',
 'я смотрю кино про себя в котором',
 'выхожу в окно будто на красную дорожку',
 'я везу в своей груди как собачье молоко',
 'этот рэпер был конкретно потливый',
 'а гробовщик позеленел как щи щавелевые',
 'я пакую твою боль в этот аудиошифр',
 'небо зашторила шершавая короста',
 'я вернулся в твою голову как в старые ботинки husky',
 'проповедь для девочек и мальчиков в трико',
 'он выглядел в гробу как котлета с подливой',
 'я возвращаюсь в твой чумазый городок',
 'я наблюдаю ебанутый новый день очередной',
 'уродливые мальчики толпой хоронят рэпера с собачьей головой wo wo',
 'заколачивали гвозди и ржавели они',
 'в одиночном кинозале через дырки в черепной',
 'улыбаясь словно',
 'поцелуй мое лицо из папье маше',
 'я купаюсь в лести как в картофельном пюр

In [57]:
len(clean_text)

76

In [56]:
len(text_prep)

73

In [95]:
len(text_prep_all)

781

In [59]:
clean_text

['я смотрю кино про себя в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино',
 'я вернулся в твою голову как в старые ботинки husky',
 'суперзвезды умирают понарошку',
 'выхожу в окно будто на красную дорожку',
 'окруженный пиздюками как какой нибудь кюре',
 'я купаюсь в лести как в картофельном пюре',
 'в одиночном кинозале через дырки в черепной',
 'я наблюдаю ебанутый новый день очередной',
 'в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино в котором',
 'я смотрю кино про себя в котором',
 'я смотрю кино про себя в к

In [74]:
# переменная для подготовленных значений
text_prep_all = []

for i in text_songs:
    text = re.findall('[А-Я][^А-Я]*', i) # создаем список фраз из текстов
    text = [a.lower() for a in text]
    text = [replace_all(a, to_replace) for a in text] # убираем лишние символы
    text = [a.rstrip().rstrip() for a in text] # удаляем пробелы
    text = [a.lstrip() for a in text]
    text = set(text) # убираем повторы
    
   # clean_text = []
  #  for idx, val in enumerate(text):
  #      if text[idx] in [' ', '']:
  #          pass
 #       else:
 #           clean_text.append(val)
    
    
   # text_prep = [phrase for phrase in text if len(phrase) > 10]
    
   # for phrase in text: # убираем короткие фразы
   #     if len(phrase) <= 10:
  #          pass
  #      else:
 #           text_prep.append(phrase)
            #print(phrase)
            #print(len(phrase))
    text_prep = []    
    for phrase in text: # убираем короткие фразы
            if len(phrase) > 10:
                text_prep.append(phrase)
            
                
    
  #  text_prep_unique = list(set(text_prep)) # убираем повторы
    text_prep_str = '\n'.join(text_prep) # переносим текст по строчкам
   # text_prep_all.append(text_prep_str)
    
    if len(text_prep_str) < 1:
        pass
    else:
        text_prep_all.append(text_prep_str)

   # clean_text.append(text_prep)
    
    #text_prep_all = [text for text in text_prep] # добавляем очищенные фразы в итоговый список
    
   # clean_text = []
   # for idx, val in enumerate(text_prep):
  #      if text_prep[idx] in [' ', '']:
   #         pass
   #     else:
   #         clean_text.append(val)
    
   # clean_text = []
   # for 

In [75]:
text_prep_all[8:12]

['и я их ебурю ебурю ебурб ебурю ебурю ебурю подручными средствами и в снег и в бурю\nизбавил всех от этой злоебучей женщины плохая наследственность в этом не ее вины\nфаршмачок мамулю я порционно слил в унитаз я сделал это не для себя я сделал это для вас\nменя опередила ее дочь тобишь моя мать но я с рождения знал что она та еще блядь\nмногим не понять что я делаю на этой планете многим не понять а мне похуй вы мне поверьте\nмногие не знают вообще что они сами тут делают а я знаю точно и в этом смысле я их всех делаю\nпотом пыталась меня внучка своего захуярить что бы я ее не захуярил когда смогу захуярить\nбогу теперь один я совсем остался отучился работаю в генетике разобрался\nи что ее надо валить при первом же случае вот такой у нас род весь злоебучий\nее мать моя бабуленька была тоже тем еще дьяволом захуярила моего батю за то что он назвал ее дьяволом\nя знаю точно точнее мне кажется некуда надо ебурить насмерть каждого недруга\nя шинканул ее на мелкий фаршмачок\nмне мама с дет

In [ ]:
python3 -m lyrics.train --songdata-file krovostok_text_8.csv --early-stopping-patience 50 --artists '*'

In [ ]:
python3 -m lyrics.train --songdata-file leps_text_4.csv --early-stopping-patience 50 --artists '*'

In [ ]:
python3 -m cli  lyrics export/$model_val/model.h5 export/$model_val/tokenizer.pickle --text 'а че теперь' --length 20

In [ ]:
python3 -m cli  lyrics export/2023-01-04T205002/model.h5 export/2023-01-04T205002/tokenizer.pickle --text 'что делать' --length 20

In [105]:
df_prepped = pd.DataFrame()


df_prepped['text'] = text_prep_all
df_prepped['artist'] = 'Кровосток'
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('~/lyrics-generator/krovostok_text_8.csv', index=False)


print(df_prepped.shape)
df_prepped

(109, 2)


,artist,text
0,Кровосток,бросается типа убрать пытается\nтакие как мы н...
1,Кровосток,с реально отрубленной головой\nоткрывает свой ...
2,Кровосток,ущипнул себя сильно за ухо я не сплю такое быв...
3,Кровосток,родился сын умерла мать\nпоказали мне пизду\nн...
4,Кровосток,не знаю как вы а я жизнью своей доволен\nразде...
...,...,...
104,Кровосток,я родился заново\nразглядывая амбивалентно\nи ...
105,Кровосток,есть ещё в подлунном мире потребитель радуги\n...
106,Кровосток,внутри меня живёт крот\nвыедает глазки солнышк...
107,Кровосток,что не станет тянуть до суда человечьего\nдреб...


In [80]:
len(df_prepped.iloc[9].text)

1515

In [93]:
df_prepped.head(30)

,artist,text
0,Кровосток,бросается типа убрать пытается\nтакие как мы н...
1,Кровосток,с реально отрубленной головой\nоткрывает свой ...
2,Кровосток,ущипнул себя сильно за ухо я не сплю такое быв...
3,Кровосток,родился сын умерла мать\nпоказали мне пизду\nн...
4,Кровосток,не знаю как вы а я жизнью своей доволен\nразде...
5,Кровосток,быть плохим значит быть запоминай красавица\nб...
6,Кровосток,вы любители\nкака течет под штаниной на\nвесна...
7,Кровосток,отложив пекарь\nты скажешь что я ебнулся\nхрус...
8,Кровосток,и я их ебурю ебурю ебурб ебурю ебурю ебурю под...
9,Кровосток,леха вернулся в\nон раздавлен уже пол года тор...
